## Previous steps:

Before executing the parameterization code, there are some things to take into account:

1. In the original email, the structure `cbm2-WT-v3.pdb` had some problems: mixed names between chains `A` and `X`, `TER` lines where they do not correspond, etc.

I've cleaned the structure of the unnecesary `TER` lines, homogeneized the chains and `Repaired` it with Foldx. This is not shown here but a complete tutorial on how to do this can be found (here)[https://github.com/leandroradusky/pyFoldX/blob/master/notebooks/StructureUsage.ipynb] (Repairing a Structure section).

2. I have put the repaired structure with the sugar in `data/cbm2-WT-v3_Repair.pdb` and without it in `data/cbm2-WT-v3_NoSugar.pdb`.

3. I've put the big sugar in a separate file, `sugar.pdb` with all the lines to parameterize it one monosaccharide a time.

Let's start!

In [1]:
# Some pyFoldX imports
from pyfoldx.handlers.fileHandler import FileHandler
from pyfoldx.structure import Structure
from pyfoldx.paramx import parameterize, JSonMolecule

## Parameterizing the monosaccharide 0GA

In [2]:
# We load the structure of the whole sugar
st = Structure("sugar", path="data/sugar.pdb")

In [3]:
# Identifying 0GA Residues
for chain in st.data.keys():
    for residue in st.data[chain]:
        #print( st.data[chain][residue].code )
        if st.data[chain][residue].code == "0GA":
            print( st.data[chain][residue] )

Residue(0GA,8,A,False)
Residue(0GA,9,A,False)


In [4]:
# We keep the lines of one 0GA residue
lines_0GA = st.data["A"][8].toPdb().split("\n")

In [5]:
lines_0GA

['ATOM   3161  O5  0GA A   8      10.175  48.870  30.962  1.00  0.12           O  ',
 'ATOM   3162  C6  0GA A   8       7.996  48.015  31.639  1.00 -0.06           C  ',
 'ATOM   3163  O6  0GA A   8       8.158  47.624  30.271  1.00  0.11           O  ',
 'ATOM   3164  C3  0GA A   8       9.922  51.357  32.401  1.00 -0.08           C  ',
 'ATOM   3165  C2  0GA A   8      11.091  51.113  31.430  1.00 -0.10           C  ',
 'ATOM   3166  C1  0GA A   8      11.430  49.624  31.175  1.00 -0.12           C  ',
 'ATOM   3167  O4  0GA A   8       7.791  50.589  33.199  1.00  0.12           O  ',
 'ATOM   3168  O3  0GA A   8       9.502  52.725  32.242  1.00  0.19           O  ',
 'ATOM   3169  O2  0GA A   8      12.235  51.794  31.974  1.00  0.22           O  ',
 'ATOM   3170  C5  0GA A   8       9.135  48.975  32.011  1.00 -0.06           C  ',
 'ATOM   3171  C4  0GA A   8       8.717  50.452  32.108  1.00 -0.06           C  ']

## Important

In the original parameterization, pyFoldX didn't existed yet. In the `data` folder I've left the mappings that I've used to parameterize the molecules then. Now, pyFoldX have names to parameterize and don't have to choose complicated names.

You can take a look at all the atoms in the supplementary material of the pyFoldX paper, but the names are these: O_hydroxyl, O_ring, O_minus, O_carboxamide, N_amino, N_guanidino, N_imidazol_plus, N_imidazol_minus, N_pyrazole, N_amide, C_ring_not_arom, C_single_link, C_double_link, C_triple_link.

Given these names and how the molecule looks like, these are the mappings:

In [6]:
atomMappingsDict = {"O5":"O_ring",
                    "C6":"C_single_link",
                    "O6":"O_hydroxyl",
                    "C3":"C_ring_not_arom",
                    "C2":"C_ring_not_arom",
                    "C1":"C_ring_not_arom",
                    "O4":"O_hydroxyl",
                    "O3":"O_hydroxyl",
                    "O2":"O_hydroxyl",
                    "C5":"C_ring_not_arom",
                    "C4":"C_ring_not_arom",
                    }

In [7]:
# Now we can parameterize the molecule
newMol = parameterize( lines_0GA, atomMappingsDict )

Mappings loaded:
Atom O5 mapped to atom ('O4*', 'A')
Atom C6 mapped to atom ('CG2', 'THR')
Atom O6 mapped to atom ('OG', 'SER')
Atom C3 mapped to atom ('CG', 'PRO')
Atom C2 mapped to atom ('CG', 'PRO')
Atom C1 mapped to atom ('CG', 'PRO')
Atom O4 mapped to atom ('OG', 'SER')
Atom O3 mapped to atom ('OG', 'SER')
Atom O2 mapped to atom ('OG', 'SER')
Atom C5 mapped to atom ('CG', 'PRO')
Atom C4 mapped to atom ('CG', 'PRO')


In [8]:
# And we can save it to a file in the molecules folder (important that is there)
FileHandler.writeLine("molecules/0GA.json", newMol.toJson())


True

### Note!! 

Here I'm just parameterizing one monosacharide, 0GA. You have all the lists that I've used in the `data` folder. I didn't have time to parameterize them all since I have to see all the mappings. This means that in the following code, what FoldX will see is only this monosaccaride. To make foldX recognize the whole sugar you will have to repeat this process with all the monosaccharides!!

## Computing the ddGs

(In this section, I'm having trouble in my local... If I execute this very lines in the python console it works good, I think is a setting related to the jupyter notebooks, I will see what happens. )

Said this, you can compute energies as follows:

In [18]:
# Load the binded structure
st_protein_binded = Structure("cbm2", path="data/cbm2-WT-v3_Repair.pdb")

In [10]:
# Load the non-binded structure
st_protein_not_binded = Structure("cbm2", path="data/cbm2-WT-v3_Repair.pdb")

In [ ]:
# Mutate the binded structure. The number of runs return just one try, but if you run more you will get better results
# the mutations (1 or more) are returned in an ensemble object. See the pyfoldx documentation for more information
# But the thing you are interested in is the energy, which is a pandas dataframe containing the energy of run.
energies, mutBindedEnsemble, wtBindedEnsemble = st_protein_binded.mutate(mutations="DX121L;", number_of_runs=1)

### Note!!

This example is valid for the mutation DX121L. Residue D, in the chain X, at pos 121, mutated to L.

You will have to run the same for several mutations. To make double mutations just separate them by comma: D121L, D122L.

In [ ]:
# Doing the same with the non-binded structure
energies, mutNotBindedEnsemble, wtNotBindedEnsemble = st_protein_not_binded.mutate(mutations="DX121L;")

In [ ]:
# You can get the total energy of the mutation in the binded structure like this.
mutBindedEnsemble.getTotalEnergy().total[0]

In [ ]:
# And then the same for the non-binded structure
mutNotBindedEnsemble.getTotalEnergy().total[0]

In [ ]:
# Substracting these two, you can have the ddG for this single mutation